# Modelo de programación lineal

## Entrada de datos: Instancia 3x150x1.5x1

In [1]:
import pandas as pd
import gurobipy as grb

In [2]:
datafile="3x150x1.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x0cf74132
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3264900 rows and 190353 columns (presolve time = 11s) ...
Presolve removed 3295944 rows and 190371 columns (presolve time = 15s) ...
Presolve removed 3313825

## Entrada de datos: Instancia 3x150x1.5x2

In [4]:
datafile="3x150x1.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xdc77a3cc
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3251396 rows and 190000 columns (presolve time = 11s) ...
Presolve removed 3279382 rows and 190086 columns (presolve time = 15s) ...
Presolve removed 3293511 rows and 190086 columns (presolve time = 20s) ...
Presolve removed 3304929 rows and 190086 columns (presolve time =

## Entrada de datos: Instancia 3x150x1.5x3

In [6]:
datafile="3x150x1.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [7]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x247459ab
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3284409 rows and 191010 columns (presolve time = 10s) ...
Presolve removed 3307793 rows and 191097 columns (presolve time = 15s) ...
Presolve removed 3327927 rows and 191097 columns (presolve time = 20s) ...
Presolve removed 3342454 rows and 191097 columns (presolve time =

## Entrada de datos: Instancia 3x150x1.5x4

In [8]:
datafile="3x150x1.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [9]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x07c31399
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3255944 rows and 189672 columns (presolve time = 11s) ...
Presolve removed 3281834 rows and 189708 columns (presolve time = 15s) ...
Presolve removed 3297422 rows and 189708 columns (presolve time = 20s) ...
Presolve removed 3310456 rows and 189708 columns (presolve time =

## Entrada de datos: Instancia 3x150x1.5x5

In [10]:
datafile="3x150x1.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [11]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x62bfcd56
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3269710 rows and 190230 columns (presolve time = 12s) ...
Presolve removed 3292723 rows and 190290 columns (presolve time = 15s) ...
Presolve removed 3308800 rows and 190290 columns (presolve time = 20s) ...
Presolve removed 3321591 rows and 190290 columns (presolve time =

 43490 14410  137.00000   80   51  136.00000  137.00000  0.74%   127  506s
 44261 14757  137.00000   80  117  136.00000  137.00000  0.74%   128  510s
 46058 14740  137.00000   74   67  136.00000  137.00000  0.74%   131  517s
 46623 14824     cutoff   87       136.00000  137.00000  0.74%   132  524s
 47196 14949  137.00000   73   71  136.00000  137.00000  0.74%   133  528s
 48144 14733  137.00000   82   55  136.00000  137.00000  0.74%   134  533s
 48352 14782     cutoff  100       136.00000  137.00000  0.74%   135  545s
 49607 14960     cutoff   97       136.00000  137.00000  0.74%   136  557s
 50555 15020  137.00000   87   70  136.00000  137.00000  0.74%   136  561s
 51639 15108  137.00000   85   49  136.00000  137.00000  0.74%   138  566s
 52900 14931     cutoff   92       136.00000  137.00000  0.74%   139  577s
 53661 15526  137.00000   98  102  136.00000  137.00000  0.74%   140  587s
 56038 14977  137.00000   90   82  136.00000  137.00000  0.74%   142  592s
 57080 15025     cutoff  

 153889 30736  137.00000   93   84  136.00000  137.00000  0.74%   192 1257s
 154916 31002     cutoff   95       136.00000  137.00000  0.74%   192 1262s
 156009 31245  137.00000   84  115  136.00000  137.00000  0.74%   192 1268s
 157106 31447  137.00000   82  185  136.00000  137.00000  0.74%   192 1273s
 157961 31718  137.00000   88   93  136.00000  137.00000  0.74%   193 1277s
 158967 31903  137.00000   93   93  136.00000  137.00000  0.74%   193 1282s
 160026 32066  137.00000   79  100  136.00000  137.00000  0.74%   193 1287s
 160932 32277     cutoff  124       136.00000  137.00000  0.74%   193 1291s
 161921 32470  137.00000   78   88  136.00000  137.00000  0.74%   193 1296s
 162987 32687  137.00000   92  144  136.00000  137.00000  0.74%   194 1300s
 165041 32959     cutoff   87       136.00000  137.00000  0.74%   194 1309s
 165873 33043  137.00000   84   58  136.00000  137.00000  0.74%   194 1320s
 167140 33458     cutoff  104       136.00000  137.00000  0.74%   194 1328s
 168156 3363

 263498 51338     cutoff   94       136.00000  137.00000  0.74%   205 1986s
 264464 51624  137.00000   91  103  136.00000  137.00000  0.74%   204 1990s
 266694 51993  137.00000   99   35  136.00000  137.00000  0.74%   204 1997s
 267771 52147  137.00000   96   56  136.00000  137.00000  0.74%   204 2001s
 268649 52283  137.00000  103   90  136.00000  137.00000  0.74%   204 2005s
 270418 52488     cutoff   80       136.00000  137.00000  0.74%   204 2012s
 271148 52545  137.00000   91   91  136.00000  137.00000  0.74%   204 2015s
 272688 52716  137.00000  101   57  136.00000  137.00000  0.74%   204 2023s
 273413 52880  137.00000   90  106  136.00000  137.00000  0.74%   204 2029s
 274308 52967     cutoff   80       136.00000  137.00000  0.74%   204 2045s
 274642 53497  137.00000  122   64  136.00000  137.00000  0.74%   204 2052s
 275980 53834  137.00000   85   51  136.00000  137.00000  0.74%   204 2059s
 277132 54081     cutoff   89       136.00000  137.00000  0.74%   204 2065s
 278023 5449

 364308 69689     cutoff   85       136.00000  137.00000  0.74%   208 2729s
 365298 69733     cutoff   93       136.00000  137.00000  0.74%   208 2735s
 366277 69842  137.00000   92  136  136.00000  137.00000  0.74%   208 2741s
 367119 69881  137.00000   77   48  136.00000  137.00000  0.74%   208 2756s
 367312 70000  137.00000   78   54  136.00000  137.00000  0.74%   208 2762s
 368531 69990  137.00000   86   55  136.00000  137.00000  0.74%   208 2768s
 369711 70023     cutoff   98       136.00000  137.00000  0.74%   208 2774s
 370573 70081     cutoff   92       136.00000  137.00000  0.74%   209 2780s
 371511 70214  137.00000   76   74  136.00000  137.00000  0.74%   209 2786s
 372506 70337  137.00000   94   66  136.00000  137.00000  0.74%   209 2792s
 373481 70439  137.00000   87   84  136.00000  137.00000  0.74%   209 2798s
 374350 70499  137.00000   92  122  136.00000  137.00000  0.74%   209 2804s
 375248 70669  137.00000   88  115  136.00000  137.00000  0.74%   209 2810s
 376296 7067

  Projected implied bound: 2
  Clique: 74
  MIR: 388
  StrongCG: 216
  Flow cover: 240
  GUB cover: 49
  Zero half: 71
  RLT: 14
  Relax-and-lift: 10

Explored 459940 nodes (102240857 simplex iterations) in 3602.12 seconds
Thread count was 6 (of 6 available processors)

Solution count 8: 136 135 132 ... -0

Time limit reached
Best objective 1.360000000000e+02, best bound 1.370000000000e+02, gap 0.7353%


## Entrada de datos: Instancia 3x150x2x1

In [12]:
datafile="3x150x2x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [13]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xa78556e6
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3308378 rows and 192777 columns (presolve time = 11s) ...
Presolve removed 3337198 rows and 192813 columns (presolve time = 15s) ...
Presolve removed 3348513 rows and 192813 columns (presolve time = 20s) ...
Presolve removed 3356854 rows and 192813 columns (presolve time =

     0     0  121.40560    0  751  118.00000  121.40560  2.89%     -  185s
     0     0  121.40538    0  763  118.00000  121.40538  2.89%     -  186s
     0     0  121.34600    0  678  118.00000  121.34600  2.84%     -  190s
     0     0  121.34217    0  673  118.00000  121.34217  2.83%     -  191s
     0     0  121.34057    0  660  118.00000  121.34057  2.83%     -  191s
     0     0  121.33995    0  677  118.00000  121.33995  2.83%     -  192s
     0     0  121.33986    0  697  118.00000  121.33986  2.83%     -  192s
     0     0  121.32689    0  677  118.00000  121.32689  2.82%     -  194s
     0     0  121.32471    0  705  118.00000  121.32471  2.82%     -  195s
     0     0  121.32419    0  714  118.00000  121.32419  2.82%     -  195s
     0     0  121.32381    0  747  118.00000  121.32381  2.82%     -  196s
     0     0  121.31637    0  753  118.00000  121.31637  2.81%     -  198s
     0     0  121.31499    0  757  118.00000  121.31499  2.81%     -  199s
     0     0  121.31463  

  6518  6492  120.82452   55  743  118.00000  120.82452  2.39%   236  890s
  6521  6494  120.80266  111  732  118.00000  120.80266  2.38%   236  896s
  6523  6496  120.79494  135  726  118.00000  120.79494  2.37%   236  900s
  6526  6498  120.78499  208  722  118.00000  120.78499  2.36%   236  907s
  6528  6499  120.33606  617  749  118.00000  120.78267  2.36%   236  910s
  6530  6500  120.29008  845  686  118.00000  120.76775  2.35%   236  915s
  6532  6502  120.54228  553  791  118.00000  120.75995  2.34%   236  922s
  6534  6503  120.65417  376  752  118.00000  120.75444  2.33%   235  928s
  6535  6504  120.66492  357  805  118.00000  120.74752  2.33%   235  930s
  6537  6505  120.29511  879  820  118.00000  120.74338  2.32%   235  935s
  6540  6507  120.62093  358  766  118.00000  120.73673  2.32%   235  942s
  6542  6508  120.45200  700  750  118.00000  120.73421  2.32%   235  946s
  6544  6510  120.24825  939  705  118.00000  120.71755  2.30%   235  951s
  6546  6511  120.54418  

  6757  6655  120.50177  338  407  118.00000  120.50177  2.12%   347 1442s
  6758  6656  120.26475  837  379  118.00000  120.50177  2.12%   347 1446s
  6760  6657  120.48725  453  533  118.00000  120.50177  2.12%   347 1453s
  6761  6658  120.26015  909  567  118.00000  120.50177  2.12%   347 1457s
  6762  6659  120.50177   81  606  118.00000  120.50177  2.12%   347 1462s
  6763  6659  120.50177  446  669  118.00000  120.50177  2.12%   347 1465s
  6765  6661  120.21719  822  653  118.00000  120.50177  2.12%   347 1470s
  6767  6662  120.16603 1035  573  118.00000  120.50177  2.12%   347 1475s
  6769  6663  120.38773  728  605  118.00000  120.50177  2.12%   346 1481s
  6771  6665  120.50177  152  580  118.00000  120.50177  2.12%   346 1485s
  6773  6666  120.50177   41  602  118.00000  120.50177  2.12%   346 1491s
  6775  6667  120.50177  190  642  118.00000  120.50177  2.12%   346 1496s
  6777  6669  120.50177  567  640  118.00000  120.50177  2.12%   346 1500s
  6779  6670  120.37475  

  6985  6810  120.17292 1015  827  118.00000  120.34260  1.99%   404 2006s
  6987  6811  120.34178  876  848  118.00000  120.34178  1.98%   404 2010s
  6990  6813  120.34028  360  836  118.00000  120.34028  1.98%   403 2017s
  6992  6815  120.12803  973  899  118.00000  120.33955  1.98%   403 2022s
  6994  6816  120.31846  721  886  118.00000  120.33906  1.98%   403 2026s
  6996  6817  120.33858  398  905  118.00000  120.33858  1.98%   403 2030s
  6998  6819  120.33778  348  901  118.00000  120.33778  1.98%   403 2035s
  7000  6820  120.33709  549  903  118.00000  120.33709  1.98%   403 2041s
  7002  6821  120.33649  379  895  118.00000  120.33649  1.98%   403 2046s
  7004  6823  120.33569  142  925  118.00000  120.33569  1.98%   403 2052s
  7006  6824  120.19931  859  905  118.00000  120.33535  1.98%   402 2055s
  7009  6826  120.33467   12  927  118.00000  120.33467  1.98%   402 2061s
  7011  6827  120.33438  355  941  118.00000  120.33438  1.98%   402 2065s
  7014  6829  120.33383  

  7216  6968  120.28058  655  701  118.00000  120.28058  1.93%   473 2556s
  7218  6970  120.28058   55  697  118.00000  120.28058  1.93%   473 2561s
  7220  6971  120.28058  694  726  118.00000  120.28058  1.93%   473 2567s
  7222  6972  120.24957  737  741  118.00000  120.28058  1.93%   473 2571s
  7224  6974  120.28058  127  822  118.00000  120.28058  1.93%   472 2576s
  7226  6975  120.28058  208  803  118.00000  120.28058  1.93%   472 2580s
  7228  6976  120.28058  617  847  118.00000  120.28058  1.93%   472 2585s
  7230  6978  120.28058  845  869  118.00000  120.28058  1.93%   472 2592s
  7232  6979  120.28041  553  858  118.00000  120.28041  1.93%   472 2597s
  7234  6980  120.27817  376  928  118.00000  120.27817  1.93%   472 2602s
  7236  6982  120.27696  331  908  118.00000  120.27696  1.93%   472 2607s
  7238  6983  120.27624  705  936  118.00000  120.27624  1.93%   472 2610s
  7241  6985  120.27587   67  941  118.00000  120.27587  1.93%   471 2615s
  7244  6987  120.24825  

## Entrada de datos: Instancia 3x150x2x2

In [14]:
datafile="3x150x2x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x274879d1
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3315844 rows and 192666 columns (presolve time = 11s) ...
Presolve removed 3339849 rows and 192711 columns (presolve time = 15s) ...
Presolve removed 3352029 rows and 192711 columns (presolve time = 20s) ...
Presolve removed 3361817 rows and 192711 columns (presolve time = 

 28036 13026  119.00000  197  189  118.00000  119.00000  0.85%   258  468s
 28807 12912     cutoff  216       118.00000  119.00000  0.85%   263  481s
 30052 12553     cutoff  216       118.00000  119.00000  0.85%   269  494s
 30547 12616  119.00000  187  200  118.00000  119.00000  0.85%   271  510s
 31377 12482  119.00000  216  216  118.00000  119.00000  0.85%   276  522s
 32691 12124     cutoff  196       118.00000  119.00000  0.85%   280  535s
 33716 11843  119.00000  214  201  118.00000  119.00000  0.85%   283  547s
 34384 11680  119.00000  200  189  118.00000  119.00000  0.85%   287  565s
 34946 11615     cutoff  211       118.00000  119.00000  0.85%   289  578s
 35663 11510  119.00000  194  131  118.00000  119.00000  0.85%   292  591s
 37058 11355     cutoff  203       118.00000  119.00000  0.85%   292  604s
 37850 11315  119.00000  191  201  118.00000  119.00000  0.85%   295  616s
 38504 11099  119.00000  196  111  118.00000  119.00000  0.85%   297  621s
 38508 11102  119.00000  

 99944 38882  119.00000  131  147  118.00000  119.00000  0.85%   302 1498s
 100765 39658  119.00000  170  178  118.00000  119.00000  0.85%   302 1507s
 101799 40274  119.00000  148  125  118.00000  119.00000  0.85%   303 1517s
 102572 40916  119.00000  255   83  118.00000  119.00000  0.85%   303 1528s
 103429 41992  119.00000  170   86  118.00000  119.00000  0.85%   304 1538s
 104769 42615  119.00000  147  161  118.00000  119.00000  0.85%   303 1546s
 105536 43272  119.00000  150  127  118.00000  119.00000  0.85%   304 1555s
 106418 43800  119.00000  156   90  118.00000  119.00000  0.85%   304 1563s
 107128 44111     cutoff  144       118.00000  119.00000  0.85%   304 1578s
 107542 44714  119.00000  102  111  118.00000  119.00000  0.85%   305 1589s
 108308 45476  119.00000  148  121  118.00000  119.00000  0.85%   305 1599s
 109430 46124     cutoff  204       118.00000  119.00000  0.85%   305 1609s
 110183 46478  119.00000  115  143  118.00000  119.00000  0.85%   306 1623s
 110596 46908

## Entrada de datos: Instancia 3x150x2x3

In [16]:
datafile="3x150x2x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x79de82a0
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3300664 rows and 192078 columns (presolve time = 11s) ...
Presolve removed 3331027 rows and 192132 columns (presolve time = 15s) ...
Presolve removed 3345322 rows and 192132 columns (presolve time = 20s) ...
Presolve removed 3355652 rows and 192132 columns (presolve time =

## Entrada de datos: Instancia 3x150x2x4

In [18]:
datafile="3x150x2x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [19]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xaa718daf
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3299390 rows and 192396 columns (presolve time = 11s) ...
Presolve removed 3324093 rows and 192438 columns (presolve time = 15s) ...
Presolve removed 3334029 rows and 192438 columns (presolve time = 20s) ...
Presolve removed 3341881 rows and 192438 columns (presolve time =

## Entrada de datos: Instancia 3x150x2x5

In [20]:
datafile="3x150x2x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [21]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x260a2906
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3310942 rows and 192342 columns (presolve time = 10s) ...
Presolve removed 3350502 rows and 192387 columns (presolve time = 15s) ...
Presolve removed 3367094 rows and 192387 columns (presolve time = 20s) ...
Presolve removed 3377088 rows and 192387 columns (presolve time =

  6986  5592  116.91667  145  165  115.00000  118.77613  3.28%   426  495s
  7455  6042  116.91667  223  213  115.00000  118.77613  3.28%   411  500s
  8522  6904  117.00000   38  193  115.00000  118.72071  3.24%   378  510s
  9002  7171  118.49599   32  373  115.00000  118.70581  3.22%   370  516s
  9269  7414  118.41565   36  329  115.00000  118.70581  3.22%   369  521s
  9516  7666  117.50000   45  175  115.00000  118.70581  3.22%   368  526s
  9804  7871  117.00000   64  170  115.00000  118.70581  3.22%   368  552s
 10017  8139  117.00000   82  235  115.00000  118.70581  3.22%   368  559s
 10311  8804  117.00000   82  297  115.00000  118.70581  3.22%   363  565s
 11192  9451  117.00000  100  236  115.00000  118.70581  3.22%   348  572s
 11974  9935  116.66667  126  183  115.00000  118.70581  3.22%   338  579s
 12611  9936  116.00000  318  281  115.00000  118.70581  3.22%   333  596s
 12614  9938  116.00000  112  401  115.00000  118.46768  3.02%   333  601s
 12620  9942  117.10377  

 68027 33167  116.00000  130  154  115.00000  116.00000  0.87%   191 1294s
 68696 33481  116.00000  117  153  115.00000  116.00000  0.87%   192 1300s
 69373 33573  116.00000  114  182  115.00000  116.00000  0.87%   193 1306s
 69780 34133  116.00000  154  127  115.00000  116.00000  0.87%   194 1312s
 70689 34399  116.00000  134   98  115.00000  116.00000  0.87%   195 1324s
 71433 34826  116.00000  168  108  115.00000  116.00000  0.87%   196 1331s
 72393 34963  116.00000  141  150  115.00000  116.00000  0.87%   197 1338s
 73024 35121     cutoff  143       115.00000  116.00000  0.87%   197 1345s
 74275 35169  116.00000  118  141  115.00000  116.00000  0.87%   198 1352s
 75324 35194     cutoff  184       115.00000  116.00000  0.87%   199 1359s
 76652 35215     cutoff  166       115.00000  116.00000  0.87%   199 1365s
 77521 35378  116.00000  180  112  115.00000  116.00000  0.87%   200 1376s
 79101 34973     cutoff  165       115.00000  116.00000  0.87%   200 1381s
 79313 35140  116.00000  

 162962 54623  116.00000  181  141  115.00000  116.00000  0.87%   225 1998s
 163689 55047  116.00000  155  117  115.00000  116.00000  0.87%   226 2005s
 164209 55626  116.00000  163   97  115.00000  116.00000  0.87%   226 2011s
 164886 56180  116.00000  188  161  115.00000  116.00000  0.87%   227 2017s
 165809 56653  116.00000  103  104  115.00000  116.00000  0.87%   227 2024s
 166455 57271  116.00000  139  131  115.00000  116.00000  0.87%   228 2030s
 167187 57522  116.00000  199  175  115.00000  116.00000  0.87%   228 2040s
 167524 58104  116.00000  122  143  115.00000  116.00000  0.87%   228 2049s
 168350 58594     cutoff  173       115.00000  116.00000  0.87%   229 2056s
 169174 58996 infeasible  134       115.00000  116.00000  0.87%   230 2064s
 169891 59602  116.00000  155  156  115.00000  116.00000  0.87%   230 2072s
 170781 60170  116.00000  179  132  115.00000  116.00000  0.87%   230 2080s
 171855 60373     cutoff  180       115.00000  116.00000  0.87%   231 2089s
 172297 6089

 262114 92173  116.00000  200  128  115.00000  116.00000  0.87%   243 2771s
 262817 92750  116.00000  153  118  115.00000  116.00000  0.87%   243 2776s
 263849 92915  116.00000  200  132  115.00000  116.00000  0.87%   243 2786s
 264217 93315     cutoff  214       115.00000  116.00000  0.87%   243 2791s
 265091 93563     cutoff  139       115.00000  116.00000  0.87%   243 2800s
 265479 94070  116.00000  174  128  115.00000  116.00000  0.87%   243 2806s
 266661 94536  116.00000  173  102  115.00000  116.00000  0.87%   243 2812s
 267934 94810  116.00000  184  140  115.00000  116.00000  0.87%   243 2820s
 268729 95096  116.00000  199  126  115.00000  116.00000  0.87%   243 2825s
 270021 95403     cutoff  183       115.00000  116.00000  0.87%   243 2835s
 270939 95967     cutoff  218       115.00000  116.00000  0.87%   243 2841s
 272511 96083  116.00000  190  116  115.00000  116.00000  0.87%   243 2846s
 273033 96323  116.00000  194  112  115.00000  116.00000  0.87%   243 2851s
 274297 9663

 362301 129000     cutoff  196       115.00000  116.00000  0.87%   244 3471s
 362780 130153  116.00000  198   51  115.00000  116.00000  0.87%   244 3479s
 364334 130642  116.00000  205  132  115.00000  116.00000  0.87%   243 3485s
 365015 131535  116.00000  228  136  115.00000  116.00000  0.87%   243 3491s
 366342 132199  116.00000  241  109  115.00000  116.00000  0.87%   243 3497s
 367546 132670  116.00000  208  138  115.00000  116.00000  0.87%   243 3503s
 368480 132994  116.00000  169  162  115.00000  116.00000  0.87%   243 3509s
 369187 133266  116.00000  174  130  115.00000  116.00000  0.87%   243 3514s
 370459 133466     cutoff  214       115.00000  116.00000  0.87%   243 3520s
 371216 133683  116.00000  220  136  115.00000  116.00000  0.87%   243 3525s
 372297 134058     cutoff  190       115.00000  116.00000  0.87%   243 3531s
 373431 134125     cutoff  229       115.00000  116.00000  0.87%   243 3542s
 374100 134376  116.00000  218  151  115.00000  116.00000  0.87%   243 3551s

## Entrada de datos: Instancia 3x150x2.5x1

In [22]:
datafile="3x150x2.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [23]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x86b3c808
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3344417 rows and 194124 columns (presolve time = 10s) ...
Presolve removed 3363899 rows and 194151 columns (presolve time = 15s) ...
Presolve removed 3371933 rows and 194151 columns (presolve time = 20s) ...
Presolve removed 3378170 rows and 194151 columns (presolve time = 

## Entrada de datos: Instancia 3x150x2.5x2

In [24]:
datafile="3x150x2.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [25]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xcf994e67
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3367055 rows and 194769 columns (presolve time = 10s) ...
Presolve removed 3385080 rows and 194835 columns (presolve time = 15s) ...
Presolve removed 3393406 rows and 194835 columns (presolve time = 20s) ...
Presolve removed 3400126 rows and 194835 columns (presolve time =

     0     0  111.04025    0  607  108.00000  111.04025  2.82%     -  132s
     0     0  111.03949    0  590  108.00000  111.03949  2.81%     -  133s
     0     0  111.03907    0  570  108.00000  111.03907  2.81%     -  133s
     0     0  111.03042    0  585  108.00000  111.03042  2.81%     -  135s
     0     0  111.02730    0  597  108.00000  111.02730  2.80%     -  135s
     0     0  111.02589    0  584  108.00000  111.02589  2.80%     -  136s
     0     0  111.02555    0  622  108.00000  111.02555  2.80%     -  136s
     0     0  111.01053    0  594  108.00000  111.01053  2.79%     -  138s
     0     0  111.00603    0  598  108.00000  111.00603  2.78%     -  139s
     0     0  111.00560    0  574  108.00000  111.00560  2.78%     -  139s
     0     0  110.99972    0  558  108.00000  110.99972  2.78%     -  140s
     0     0  110.99730    0  577  108.00000  110.99730  2.78%     -  141s
     0     0  110.99688    0  577  108.00000  110.99688  2.77%     -  141s
     0     0  110.98527  

  4767  4574  110.50934   34  634  108.00000  110.50934  2.32%   227  606s
  4769  4575  110.36529  173  574  108.00000  110.50406  2.32%   227  610s
  4773  4578  109.03381  691  669  108.00000  110.49667  2.31%   227  615s
  4777  4580  109.84207  318  668  108.00000  110.49399  2.31%   226  620s
  4781  4583  110.49187   56  634  108.00000  110.49187  2.31%   226  625s
  4785  4586  110.45729  175  643  108.00000  110.48881  2.30%   226  630s
  4791  4590  109.24942  603  637  108.00000  110.48580  2.30%   226  636s
  4795  4592  109.54904  430  668  108.00000  110.48411  2.30%   226  640s
  4800  4596  110.47052  130  678  108.00000  110.48209  2.30%   225  645s
  4804  4598  110.01978  428  711  108.00000  110.48005  2.30%   225  650s
  4808  4601  110.05937  312  661  108.00000  110.47805  2.29%   225  655s
  4813  4604  109.93367  310  616  108.00000  110.47697  2.29%   225  660s
  4818  4608  109.84207  318  665  108.00000  110.47447  2.29%   225  665s
  4822  4610  110.20880  

  6257  5371  110.26156   88  521  109.00000  110.36900  1.26%   322 1241s
  6355  5455  110.35218   92  586  109.00000  110.36900  1.26%   326 1248s
  6423  5490  110.35162   94  595  109.00000  110.36900  1.26%   328 1251s
  6481  5512  110.28713   95  491  109.00000  110.36900  1.26%   331 1255s
  6623  5646  110.35042  100  573  109.00000  110.36900  1.26%   336 1262s
  6705  5683  110.34940  103  558  109.00000  110.36900  1.26%   338 1267s
  6773  5747  110.34892  106  514  109.00000  110.36900  1.26%   341 1271s
  6859  5833  110.34634  109  573  109.00000  110.36900  1.26%   343 1275s
  7030  5912     cutoff  113       109.00000  110.36900  1.26%   346 1283s
  7127  5958  110.34329  115  482  109.00000  110.36900  1.26%   348 1287s
  7216  6033  110.33916  118  484  109.00000  110.36900  1.26%   351 1292s
  7336  6106  110.33815  120  507  109.00000  110.36900  1.26%   353 1296s
  7462  6146  110.30178  124  476  109.00000  110.36900  1.26%   355 1302s
  7549  6187  110.33132  

 25738 17432  110.13195  260  650  109.00000  110.31427  1.21%   517 2561s
 25741 17434  110.04019  459  657  109.00000  110.31350  1.21%   517 2565s
 25744 17436  110.02968  257  638  109.00000  110.31224  1.20%   517 2570s
 25747 17438  110.21392  287  641  109.00000  110.31133  1.20%   517 2575s
 25750 17440  110.08880  219  619  109.00000  110.31081  1.20%   517 2580s
 25753 17442  110.28120   95  624  109.00000  110.31011  1.20%   517 2585s
 25756 17444  110.01531  242  611  109.00000  110.30966  1.20%   517 2590s
 25759 17446  110.24114   88  696  109.00000  110.30855  1.20%   517 2596s
 25762 17448  110.24568  120  669  109.00000  110.30830  1.20%   516 2601s
 25765 17450  110.23448  138  679  109.00000  110.30810  1.20%   516 2606s
 25767 17451  110.30803   77  668  109.00000  110.30803  1.20%   516 2611s
 25770 17453  110.03061  179  673  109.00000  110.30798  1.20%   516 2616s
 25773 17455  110.19097  147  673  109.00000  110.30767  1.20%   516 2621s
 25776 17457  110.26908  

## Entrada de datos: Instancia 3x150x2.5x3

In [26]:
datafile="3x150x2.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [27]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x61f84bcd
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3357991 rows and 194757 columns (presolve time = 10s) ...
Presolve removed 3376446 rows and 194805 columns (presolve time = 15s) ...
Presolve removed 3384639 rows and 194805 columns (presolve time = 20s) ...
Presolve removed 3391541 rows and 194805 columns (presolve time =

## Entrada de datos: Instancia 3x150x2.5x4

In [28]:
datafile="3x150x2.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [29]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x1739f953
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3367663 rows and 194961 columns (presolve time = 10s) ...
Presolve removed 3381825 rows and 195003 columns (presolve time = 15s) ...
Presolve removed 3391511 rows and 195003 columns (presolve time = 20s) ...
Presolve removed 3399509 rows and 195003 columns (presolve time =

  Clique: 16
  MIR: 141
  StrongCG: 67
  Flow cover: 110
  GUB cover: 18
  Zero half: 42
  RLT: 7
  Relax-and-lift: 7

Explored 1464 nodes (1248397 simplex iterations) in 289.15 seconds
Thread count was 6 (of 6 available processors)

Solution count 10: 106 105 104 ... 10

Optimal solution found (tolerance 1.00e-04)
Best objective 1.060000000000e+02, best bound 1.060000000000e+02, gap 0.0000%


## Entrada de datos: Instancia 3x150x2.5x5

In [30]:
datafile="3x150x2.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [31]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x407209c3
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3365867 rows and 194676 columns (presolve time = 10s) ...
Presolve removed 3388102 rows and 194721 columns (presolve time = 15s) ...
Presolve removed 3396479 rows and 194721 columns (presolve time = 20s) ...
Presolve removed 3402338 rows and 194721 columns (presolve time =

## Entrada de datos: Instancia 3x150x3x1

In [32]:
datafile="3x150x3x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [33]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x3c354b97
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3380932 rows and 195591 columns (presolve time = 10s) ...
Presolve removed 3397835 rows and 195636 columns (presolve time = 15s) ...
Presolve removed 3405734 rows and 195636 columns (presolve time = 20s) ...
Presolve removed 3412162 rows and 195636 columns (presolve time = 

 10541  7318     cutoff  190        89.00000   90.33333  1.50%   206  401s
 11265  7516   90.13414  135  196   89.00000   90.33333  1.50%   210  408s
 11591  7674   90.00000  147  120   89.00000   90.33333  1.50%   214  414s
 11812  7795     cutoff  147        89.00000   90.33333  1.50%   218  420s
 11996  7930   90.25336  113  244   89.00000   90.33333  1.50%   225  427s
 12200  8159     cutoff  117        89.00000   90.33333  1.50%   229  433s
 12608  8160   90.25623  195  214   89.00000   90.33333  1.50%   237  443s
 12611  8162   90.16667  163  218   89.00000   90.33333  1.50%   237  445s
 12615  8165   90.16138  126  101   89.00000   90.33333  1.50%   237  451s
 12620  8168   90.00000   60  205   89.00000   90.33333  1.50%   237  455s
 12627  8173   90.00000  130  132   89.00000   90.33333  1.50%   237  461s
 12632  8179   90.33333   39  142   89.00000   90.33333  1.50%   243  466s
 12644  8186   90.33333   42  159   89.00000   90.33333  1.50%   243  470s
 12675  8196   90.33333  

## Entrada de datos: Instancia 3x150x3x2

In [34]:
datafile="3x150x3x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [35]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x6e15a269
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3387267 rows and 195858 columns (presolve time = 10s) ...
Presolve removed 3404672 rows and 195894 columns (presolve time = 15s) ...
Presolve removed 3412145 rows and 195894 columns (presolve time = 20s) ...
Presolve removed 3418163 rows and 195894 columns (presolve time =

## Entrada de datos: Instancia 3x150x3x3

In [36]:
datafile="3x150x3x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x8e656ab6
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3387299 rows and 195879 columns (presolve time = 10s) ...
Presolve removed 3404453 rows and 195909 columns (presolve time = 15s) ...
Presolve removed 3413257 rows and 195909 columns (presolve time = 20s) ...
Presolve removed 3420784 rows and 195909 columns (presolve time =

     0     0   95.24011    0  421   93.00000   95.24011  2.41%     -  143s
     0     0   95.24011    0  445   93.00000   95.24011  2.41%     -  143s
     0     0   95.23848    0  437   93.00000   95.23848  2.41%     -  145s
H    0     0                      94.0000000   95.23848  1.32%     -  147s
     0     0   95.23763    0  406   94.00000   95.23763  1.32%     -  147s
     0     0   95.23749    0  414   94.00000   95.23749  1.32%     -  147s
     0     0   95.23650    0  395   94.00000   95.23650  1.32%     -  148s
     0     0   95.23650    0  134   94.00000   95.23650  1.32%     -  152s
     0     0   95.23650    0  342   94.00000   95.23650  1.32%     -  153s
     0     0   95.23650    0  390   94.00000   95.23650  1.32%     -  154s
     0     0   95.23650    0  392   94.00000   95.23650  1.32%     -  156s
     0     0   95.23650    0  398   94.00000   95.23650  1.32%     -  156s
     0     0   95.23650    0  388   94.00000   95.23650  1.32%     -  157s
     0     0   95.23650  

 17239 11231   95.01450   81  436   94.00000   95.07523  1.14%   472  831s
 18029 11755     cutoff   50        94.00000   95.07472  1.14%   476  861s
 18820 12297   95.01597   69  429   94.00000   95.07374  1.14%   480  891s
 19579 12834   95.05221   45  449   94.00000   95.07288  1.14%   483  921s
 20350 13470   95.03592   76  419   94.00000   95.07240  1.14%   488  953s
 21212 13972   95.04326   77  417   94.00000   95.07205  1.14%   490  985s
 21971 14491   95.00883   74  426   94.00000   95.07166  1.14%   494 1016s
 22765 14971   95.00300   77  396   94.00000   95.07119  1.14%   498 1049s
 23527 15494   95.03926   61  450   94.00000   95.07054  1.14%   502 1082s
 24303 15966   95.03643   54  430   94.00000   95.07021  1.14%   505 1115s
 25046 16406   95.02805   52  398   94.00000   95.06985  1.14%   509 1149s
 25772 16407   95.01187  100  395   94.00000   95.06927  1.14%   514 1207s
 25777 16410   95.00292  129  437   94.00000   95.06927  1.14%   514 1210s
 25780 16412   95.05346  

## Entrada de datos: Instancia 3x150x3x4

In [38]:
datafile="3x150x3x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [39]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xe7d5049a
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3376972 rows and 195711 columns (presolve time = 11s) ...
Presolve removed 3399411 rows and 195744 columns (presolve time = 15s) ...
Presolve removed 3409545 rows and 195744 columns (presolve time = 20s) ...
Presolve removed 3416757 rows and 195744 columns (presolve time =

   821   732   92.71304   38  376   91.00000   92.77183  1.95%   717  156s
   874   768   92.71279   45  320   91.00000   92.77183  1.95%   722  160s
  1021   882   92.70185   59  415   91.00000   92.77183  1.95%   687  167s
  1060   921   92.70168   62  362   91.00000   92.77183  1.95%   694  170s
  1153  1007   92.69978   76  348   91.00000   92.77183  1.95%   723  178s
  1202  1042   92.69769   89  381   91.00000   92.77183  1.95%   723  181s
  1312  1117   92.52125   35  249   91.00000   92.77183  1.95%   738  198s
  1315  1119   92.51451   17  239   91.00000   92.77183  1.95%   736  200s
  1319  1122   92.67735   19  205   91.00000   92.77183  1.95%   734  205s
  1328  1128   92.71373   22  267   91.00000   92.76732  1.94%   729  210s
  1335  1132   92.03522   85  292   91.00000   92.71616  1.89%   725  215s
  1343  1138   92.66629   20  268   91.00000   92.66629  1.83%   721  220s
  1353  1144   92.63087   27  246   91.00000   92.63087  1.79%   715  225s
  1361  1150   92.61635  

## Entrada de datos: Instancia 3x150x3x5

In [40]:
datafile="3x150x3x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [41]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x912961cf
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3379607 rows and 195924 columns (presolve time = 10s) ...
Presolve removed 3398629 rows and 195945 columns (presolve time = 15s) ...
Presolve removed 3409326 rows and 195945 columns (presolve time = 20s) ...
Presolve removed 3417845 rows and 195945 columns (presolve time =

 27993 19471   95.49158   98  174   94.00000   95.50000  1.60%   191  318s
 28149 19765   95.48176  120  154   94.00000   95.50000  1.60%   191  320s
 28998 20050   95.50000   37  158   94.00000   95.50000  1.60%   193  325s
 29391 20196   95.42559   92  196   94.00000   95.50000  1.60%   194  336s
 30147 20434   95.50000   45  157   94.00000   95.50000  1.60%   195  345s
 30320 20705   95.50000   46  160   94.00000   95.50000  1.60%   196  350s
 31336 21238   95.39289   51  257   94.00000   95.50000  1.60%   198  367s
 31853 21206   95.00000  146  130   94.00000   95.50000  1.60%   201  371s
 32048 21601   95.00000  171  174   94.00000   95.50000  1.60%   201  376s
 32647 22041   95.48879   71  268   94.00000   95.50000  1.60%   204  383s
 33477 22302   95.00000   63  142   94.00000   95.50000  1.60%   207  388s
 34097 22569   95.00000  144  128   94.00000   95.50000  1.60%   211  398s
 34687 22977   95.00000  208  129   94.00000   95.50000  1.60%   213  403s
 35421 22984   95.50000  

 95619 50751     cutoff  126        94.00000   95.50000  1.60%   327 1327s
 96452 51316   95.00000  143  210   94.00000   95.50000  1.60%   328 1339s
 97529 51790   95.00000  143  166   94.00000   95.50000  1.60%   328 1351s
 98523 52250   95.50000   46  278   94.00000   95.50000  1.60%   328 1363s
 99491 52591   95.33333   56  240   94.00000   95.50000  1.60%   329 1374s
 100322 53058     cutoff  154        94.00000   95.50000  1.60%   329 1386s
 101277 53378   95.00000  114  143   94.00000   95.50000  1.60%   329 1397s
 102056 53849   95.00000  139  209   94.00000   95.50000  1.60%   330 1409s
 102946 54311   95.00000  189  209   94.00000   95.50000  1.60%   331 1421s
 103895 54744   95.00000   79   96   94.00000   95.50000  1.60%   331 1433s
 104912 54943   95.00000  185  153   94.00000   95.50000  1.60%   331 1444s
 105612 55489     cutoff  213        94.00000   95.50000  1.60%   332 1457s
 106545 55996     cutoff  138        94.00000   95.50000  1.60%   333 1468s
 107491 56203   9

 191722 121119   95.00000   74  124   94.00000   95.50000  1.60%   367 2760s
 192612 121873   95.00000   66  197   94.00000   95.50000  1.60%   367 2774s
 193564 122664     cutoff  140        94.00000   95.50000  1.60%   368 2788s
 194565 123435   95.00000  108  142   94.00000   95.50000  1.60%   368 2802s
 195501 124265     cutoff  171        94.00000   95.50000  1.60%   368 2816s
 196549 125045   95.00000   52  266   94.00000   95.50000  1.60%   368 2829s
 197482 125849   95.00000  141  107   94.00000   95.50000  1.60%   368 2842s
 198473 126421     cutoff  180        94.00000   95.50000  1.60%   368 2853s
 199216 126985   95.00000  158  199   94.00000   95.50000  1.60%   368 2866s
 199912 127592   95.00000   81  121   94.00000   95.50000  1.60%   369 2879s
 200675 128332   95.47222   55  323   94.00000   95.50000  1.60%   369 2892s
 201596 129052   95.00000   69  146   94.00000   95.50000  1.60%   369 2906s
 202494 129701   95.33333   51  292   94.00000   95.50000  1.60%   370 2920s

## Entrada de datos: Instancia 3x150x3.5x1

In [42]:
datafile="3x150x3.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [43]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x6ecb0503
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3405499 rows and 196851 columns (presolve time = 11s) ...
Presolve removed 3423708 rows and 196890 columns (presolve time = 15s) ...
Presolve removed 3432804 rows and 196890 columns (presolve time = 20s) ...
Presolve removed 3439693 rows and 196890 columns (presolve time =

     0     0   89.92340    0  303   88.00000   89.92340  2.19%     -   66s
     0     0   89.92261    0  305   88.00000   89.92261  2.18%     -   66s
     0     0   89.92235    0  322   88.00000   89.92235  2.18%     -   66s
     0     0   89.91932    0  287   88.00000   89.91932  2.18%     -   66s
     0     0   89.91855    0  282   88.00000   89.91855  2.18%     -   66s
     0     0   89.91836    0  281   88.00000   89.91836  2.18%     -   66s
     0     0   89.91544    0  309   88.00000   89.91544  2.18%     -   67s
     0     0   89.91513    0  341   88.00000   89.91513  2.18%     -   67s
     0     0   89.90763    0  313   88.00000   89.90763  2.17%     -   67s
     0     0   89.90709    0  302   88.00000   89.90709  2.17%     -   67s
     0     0   89.90704    0  296   88.00000   89.90704  2.17%     -   67s
     0     0   89.90626    0  284   88.00000   89.90626  2.17%     -   68s
     0     0   89.90546    0  273   88.00000   89.90546  2.17%     -   68s
     0     0   89.90458  

## Entrada de datos: Instancia 3x150x3.5x2

In [44]:
datafile="3x150x3.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [45]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xb5f649ec
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3411117 rows and 196938 columns (presolve time = 10s) ...
Presolve removed 3424049 rows and 196974 columns (presolve time = 15s) ...
Presolve removed 3430048 rows and 196974 columns (presolve time = 20s) ...
Presolve removed 3435748 rows and 196974 columns (presolve time = 

  7992  1021     cutoff   47        86.00000   87.00000  1.16%   469  208s
  8200  1110   87.00000   51  188   86.00000   87.00000  1.16%   470  215s
  8782  1165   87.00000   59  217   86.00000   87.00000  1.16%   476  222s
  9366  1183     cutoff   75        86.00000   87.00000  1.16%   478  230s
  9993  1272   87.00000   63  190   86.00000   87.00000  1.16%   484  238s
 10540  1304     cutoff   80        86.00000   87.00000  1.16%   485  246s
 11245  1277     cutoff   60        86.00000   87.00000  1.16%   489  255s
 11765  1271   87.00000   54  161   86.00000   87.00000  1.16%   492  264s
 12518  1240   87.00000   64  168   86.00000   87.00000  1.16%   494  273s
 13294  1225     cutoff   65        86.00000   87.00000  1.16%   500  283s
 14055  1243   87.00000   60  222   86.00000   87.00000  1.16%   503  295s
 14350  1253   87.00000   55  200   86.00000   87.00000  1.16%   504  300s

Cutting planes:
  Gomory: 20
  Cover: 33
  Implied bound: 1
  Projected implied bound: 2
  Clique: 

## Entrada de datos: Instancia 3x150x3.5x3

In [46]:
datafile="3x150x3.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x8df7b5c0
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 3 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3395200 rows and 196587 columns (presolve time = 10s) ...
Presolve removed 3412446 rows and 196629 columns (presolve time = 15s) ...
Presolve removed 3421157 rows and 196629 columns (presolve time = 20s) ...
Presolve removed 3427445 rows and 196629 columns (presolve time =

     0     0   89.46862    0  229   88.00000   89.46862  1.67%     -   72s
     0     0   89.44828    0  221   88.00000   89.44828  1.65%     -   72s
     0     0   89.44759    0  219   88.00000   89.44759  1.64%     -   72s
     0     0   89.44759    0  211   88.00000   89.44759  1.64%     -   72s
     0     0   89.44759    0  231   88.00000   89.44759  1.64%     -   73s
     0     0   89.44759    0  230   88.00000   89.44759  1.64%     -   73s
     0     0   89.44759    0  226   88.00000   89.44759  1.64%     -   73s
     0     0   89.44759    0  152   88.00000   89.44759  1.64%     -   73s
     0     2   89.44759    0  136   88.00000   89.44759  1.64%     -   73s
   131   152   89.39599   25  173   88.00000   89.43103  1.63%   125   75s
  1762  1412   89.00000  258  215   88.00000   89.41763  1.61%  88.4   80s
  1790  1431   89.00000   22  283   88.00000   89.39878  1.59%  87.0   85s
  1818  1454   89.37500   17   74   88.00000   89.37500  1.56%   109   91s
  1912  1489   89.37500  

## Entrada de datos: Instancia 3x150x3.5x4

In [48]:
datafile="3x150x3.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [49]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x3926114f
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3409347 rows and 196848 columns (presolve time = 10s) ...
Presolve removed 3418929 rows and 196890 columns (presolve time = 15s) ...
Presolve removed 3424318 rows and 196890 columns (presolve time = 20s) ...
Presolve removed 3429287 rows and 196890 columns (presolve time =

  8577  2452   92.00000   75  194   91.00000   92.00000  1.10%   298  165s
  9433  2554   92.00000   69  164   91.00000   92.00000  1.10%   315  171s
 10314  2634     cutoff   68        91.00000   92.00000  1.10%   331  178s
 10743  2627   92.00000   81  187   91.00000   92.00000  1.10%   334  182s
 10898  2673   92.00000   58  187   91.00000   92.00000  1.10%   341  186s
 11753  2777   92.00000   64  224   91.00000   92.00000  1.10%   360  202s
 12070  2907   92.00000   73  177   91.00000   92.00000  1.10%   363  206s
 12567  3232   92.00000   63  127   91.00000   92.00000  1.10%   367  213s
 13488  3278     cutoff   83        91.00000   92.00000  1.10%   378  219s
 13633  3390   92.00000   55  207   91.00000   92.00000  1.10%   378  226s
 14263  3734   92.00000   70  158   91.00000   92.00000  1.10%   383  234s
 15114  3810   92.00000   57  143   91.00000   92.00000  1.10%   399  252s
H15193   403                      92.0000000   92.00000  0.00%   399  252s

Cutting planes:
  Gomory

## Entrada de datos: Instancia 3x150x3.5x5

In [50]:
datafile="3x150x3.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [51]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x239be4d9
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 8.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3420303 rows and 197157 columns (presolve time = 10s) ...
Presolve removed 3439432 rows and 197181 columns (presolve time = 15s) ...
Presolve removed 3446906 rows and 197181 columns (presolve time = 20s) ...
Presolve removed 3453187 rows and 197181 columns (presolve time = 

     0     0   88.04213    0  279   86.00000   88.04213  2.37%     -   64s
     0     0   88.04190    0  269   86.00000   88.04190  2.37%     -   64s
     0     0   88.02320    0  303   86.00000   88.02320  2.35%     -   64s
     0     0   88.02235    0  304   86.00000   88.02235  2.35%     -   64s
     0     0   88.02226    0  295   86.00000   88.02226  2.35%     -   64s
     0     0   87.97219    0  277   86.00000   87.97219  2.29%     -   65s
     0     0   87.97089    0  247   86.00000   87.97089  2.29%     -   65s
     0     0   87.97086    0  250   86.00000   87.97086  2.29%     -   65s
     0     0   87.97083    0  267   86.00000   87.97083  2.29%     -   65s
     0     0   87.97083    0  235   86.00000   87.97083  2.29%     -   65s
     0     0   87.96252    0  276   86.00000   87.96252  2.28%     -   66s
     0     0   87.95247    0  290   86.00000   87.95247  2.27%     -   66s
     0     0   87.95197    0  302   86.00000   87.95197  2.27%     -   66s
     0     0   87.95177  

 28214  1453   87.01065  140  124   86.00000   87.04887  1.22%   258  338s
 29812  2378     cutoff  116        86.00000   87.03125  1.20%   250  348s
 33788  2218   87.00240  122  129   86.00000   87.03125  1.20%   228  350s
 36841   802   87.00044  108  122   86.00000   87.01234  1.18%   216  356s
 39425   195     cutoff  133        86.00000   87.00000  1.16%   205  360s

Cutting planes:
  Gomory: 15
  Cover: 171
  Projected implied bound: 1
  Clique: 30
  MIR: 287
  StrongCG: 170
  Flow cover: 127
  GUB cover: 6
  Zero half: 34
  RLT: 2
  Relax-and-lift: 13

Explored 40183 nodes (8215716 simplex iterations) in 366.60 seconds
Thread count was 6 (of 6 available processors)

Solution count 9: 86 85 85 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 8.600000000000e+01, best bound 8.600000000000e+01, gap 0.0000%


## Entrada de datos: Instancia 3x150x4x1

In [52]:
datafile="3x150x4x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [53]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xa8f1fb2d
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3429486 rows and 197805 columns (presolve time = 10s) ...
Presolve removed 3448769 rows and 197841 columns (presolve time = 15s) ...
Presolve removed 3457794 rows and 197841 columns (presolve time = 20s) ...
Presolve removed 3464162 rows and 197841 columns (presolve time =

  GUB cover: 15
  Zero half: 19
  RLT: 5
  Relax-and-lift: 5

Explored 6201 nodes (1085025 simplex iterations) in 112.77 seconds
Thread count was 6 (of 6 available processors)

Solution count 8: 81 80 79 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 8.100000079343e+01, best bound 8.100000079515e+01, gap 0.0000%


## Entrada de datos: Instancia 3x150x4x2

In [54]:
datafile="3x150x4x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [55]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x0d41b9a7
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3411471 rows and 197268 columns (presolve time = 10s) ...
Presolve removed 3428511 rows and 197298 columns (presolve time = 15s) ...
Presolve removed 3434449 rows and 197298 columns (presolve time = 20s) ...
Presolve removed 3438625 rows and 197298 columns (presolve time = 

## Entrada de datos: Instancia 3x150x4x3

In [56]:
datafile="3x150x4x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [57]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x6945ce54
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3419158 rows and 197460 columns (presolve time = 10s) ...
Presolve removed 3433122 rows and 197502 columns (presolve time = 15s) ...
Presolve removed 3440051 rows and 197502 columns (presolve time = 20s) ...
Presolve removed 3445745 rows and 197502 columns (presolve time =


Optimal solution found (tolerance 1.00e-04)
Best objective 8.100000000000e+01, best bound 8.100000000000e+01, gap 0.0000%


## Entrada de datos: Instancia 3x150x4x4

In [58]:
datafile="3x150x4x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [59]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x4eb57191
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3400794 rows and 196911 columns (presolve time = 10s) ...
Presolve removed 3419513 rows and 196947 columns (presolve time = 15s) ...
Presolve removed 3432193 rows and 196947 columns (presolve time = 20s) ...
Presolve removed 3442227 rows and 196947 columns (presolve time =

 27078  1869   83.00000   56  214   82.00000   83.00000  1.22%   639  777s
 27252  1800   83.00000   70  178   82.00000   83.00000  1.22%   642  784s
 27596  1713     cutoff   68        82.00000   83.00000  1.22%   645  791s
 27815  1625   83.00000   60  176   82.00000   83.00000  1.22%   648  799s
 28105  1518   83.00000   57  176   82.00000   83.00000  1.22%   652  807s
 28342  1373     cutoff   64        82.00000   83.00000  1.22%   657  815s
 28709  1216     cutoff   67        82.00000   83.00000  1.22%   661  823s
 29113  1040     cutoff   67        82.00000   83.00000  1.22%   664  830s
 29374   968     cutoff   63        82.00000   83.00000  1.22%   666  836s
 29573   852     cutoff   64        82.00000   83.00000  1.22%   667  845s
 29751   716   83.00000   60  201   82.00000   83.00000  1.22%   669  850s
 29983   611     cutoff   63        82.00000   83.00000  1.22%   673  855s

Cutting planes:
  Gomory: 29
  Cover: 68
  Clique: 19
  MIR: 184
  StrongCG: 127
  Flow cover: 54
 

## Entrada de datos: Instancia 3x150x4x5

In [60]:
datafile="3x150x4x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [61]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x6583359a
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3402860 rows and 196881 columns (presolve time = 10s) ...
Presolve removed 3424871 rows and 196917 columns (presolve time = 15s) ...
Presolve removed 3433351 rows and 196917 columns (presolve time = 20s) ...
Presolve removed 3439645 rows and 196917 columns (presolve time =

     0     0   86.39864    0  168   85.00000   86.39864  1.65%     -  134s
     0     0   86.39864    0  235   85.00000   86.39864  1.65%     -  134s
     0     0   86.39864    0  326   85.00000   86.39864  1.65%     -  135s
     0     0   86.39864    0  333   85.00000   86.39864  1.65%     -  136s
     0     0   86.39864    0  364   85.00000   86.39864  1.65%     -  136s
     0     0   86.39864    0  368   85.00000   86.39864  1.65%     -  136s
     0     0   86.39864    0  359   85.00000   86.39864  1.65%     -  137s
     0     0   86.39864    0  355   85.00000   86.39864  1.65%     -  137s
     0     0   86.39861    0  375   85.00000   86.39861  1.65%     -  137s
     0     0   86.39712    0  369   85.00000   86.39712  1.64%     -  137s
     0     0   86.39675    0  373   85.00000   86.39675  1.64%     -  137s
     0     0   86.38319    0  373   85.00000   86.38319  1.63%     -  138s
     0     0   86.37289    0  369   85.00000   86.37289  1.62%     -  139s
     0     0   86.37122  

## Entrada de datos: Instancia 3x150x4.5x1

In [62]:
datafile="3x150x4.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [63]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xa0c53e92
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 7.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3425803 rows and 197973 columns (presolve time = 10s) ...
Presolve removed 3444231 rows and 197991 columns (presolve time = 15s) ...
Presolve removed 3452386 rows and 197991 columns (presolve time = 20s) ...
Presolve removed 3459151 rows and 197991 columns (presolve time = 

     0     0   75.33011    0  265   74.00000   75.33011  1.80%     -   62s
     0     0   75.32542    0  255   74.00000   75.32542  1.79%     -   62s
     0     0   75.32332    0  260   74.00000   75.32332  1.79%     -   62s
     0     0   75.32250    0  274   74.00000   75.32250  1.79%     -   62s
     0     0   75.32191    0  265   74.00000   75.32191  1.79%     -   62s
     0     0   75.32173    0  268   74.00000   75.32173  1.79%     -   62s
     0     0   75.30277    0  273   74.00000   75.30277  1.76%     -   62s
     0     0   75.30066    0  271   74.00000   75.30066  1.76%     -   62s
     0     0   75.30037    0  273   74.00000   75.30037  1.76%     -   62s
     0     0   75.29217    0  288   74.00000   75.29217  1.75%     -   63s
     0     0   75.29004    0  299   74.00000   75.29004  1.74%     -   63s
     0     0   75.28902    0  302   74.00000   75.28902  1.74%     -   63s
     0     0   75.28879    0  300   74.00000   75.28879  1.74%     -   63s
     0     0   75.27622  

## Entrada de datos: Instancia 3x150x4.5x2

In [64]:
datafile="3x150x4.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [65]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x5d372901
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 9.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3428371 rows and 197931 columns (presolve time = 10s) ...
Presolve removed 3444227 rows and 197961 columns (presolve time = 15s) ...
Presolve removed 3453057 rows and 197961 columns (presolve time = 20s) ...
Presolve removed 3460178 rows and 197961 columns (presolve time = 

## Entrada de datos: Instancia 3x150x4.5x3

In [66]:
datafile="3x150x4.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [67]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x46298ba6
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3416667 rows and 197616 columns (presolve time = 10s) ...
Presolve removed 3432187 rows and 197658 columns (presolve time = 15s) ...
Presolve removed 3441127 rows and 197658 columns (presolve time = 20s) ...
Presolve removed 3448177 rows and 197658 columns (presolve time =

## Entrada de datos: Instancia 3x150x4.5x4

In [68]:
datafile="3x150x4.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [69]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x4cf4ee6d
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3426728 rows and 197751 columns (presolve time = 10s) ...
Presolve removed 3446782 rows and 197778 columns (presolve time = 15s) ...
Presolve removed 3455687 rows and 197778 columns (presolve time = 20s) ...
Presolve removed 3462612 rows and 197778 columns (presolve time =

     0     0   75.95853    0  274   73.00000   75.95853  4.05%     -   61s
H    0     0                      74.0000000   75.95853  2.65%     -   61s
     0     0   75.95796    0  263   74.00000   75.95796  2.65%     -   61s
     0     0   75.95766    0  296   74.00000   75.95766  2.65%     -   61s
     0     0   75.95739    0  268   74.00000   75.95739  2.65%     -   62s
     0     0   75.95735    0  274   74.00000   75.95735  2.65%     -   62s
     0     0   75.95562    0  271   74.00000   75.95562  2.64%     -   62s
     0     0   75.95517    0  271   74.00000   75.95517  2.64%     -   62s
     0     0   75.95453    0  262   74.00000   75.95453  2.64%     -   62s
     0     0   75.95435    0  260   74.00000   75.95435  2.64%     -   62s
     0     0   75.95377    0  272   74.00000   75.95377  2.64%     -   62s
     0     0   75.95350    0  275   74.00000   75.95350  2.64%     -   63s
     0     0   75.95298    0  265   74.00000   75.95298  2.64%     -   63s
     0     0   75.95297  

 38808  8606   75.00000   93  179   74.00000   75.00000  1.35%   422  590s
 38820  8614   75.00000  117  172   74.00000   75.00000  1.35%   421  595s
 38831  8621   75.00000  133  206   74.00000   75.00000  1.35%   421  600s
 38843  8629   75.00000  143  224   74.00000   75.00000  1.35%   421  605s
 38856  8638   75.00000  100  186   74.00000   75.00000  1.35%   421  610s
 38871  8648   75.00000  105  198   74.00000   75.00000  1.35%   421  615s
 38886  8659   75.00000  109  183   74.00000   75.00000  1.35%   422  620s
 38901  8669   75.00000  136  208   74.00000   75.00000  1.35%   422  625s
 38910  8676   75.00000  132  209   74.00000   75.00000  1.35%   418  630s
 38927  8687   75.00000  107  184   74.00000   75.00000  1.35%   417  635s

Cutting planes:
  Gomory: 15
  Cover: 19
  Clique: 19
  MIR: 253
  StrongCG: 132
  Flow cover: 103
  GUB cover: 6
  Zero half: 38
  RLT: 9

Explored 38937 nodes (16534972 simplex iterations) in 643.14 seconds
Thread count was 6 (of 6 available proce

## Entrada de datos: Instancia 3x150x4.5x5

In [70]:
datafile="3x150x4.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [71]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x2ae0f776
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3423615 rows and 197955 columns (presolve time = 10s) ...
Presolve removed 3441558 rows and 197982 columns (presolve time = 15s) ...
Presolve removed 3448699 rows and 197982 columns (presolve time = 20s) ...
Presolve removed 3453865 rows and 197982 columns (presolve time =

 38856 11857   76.00000  164  161   75.00000   76.00000  1.33%   222  379s
 39250 11986   76.00000  136  160   75.00000   76.00000  1.33%   225  394s
 40032 12038     cutoff  148        75.00000   76.00000  1.33%   229  409s
 41296 11814   76.00000  164  167   75.00000   76.00000  1.33%   235  423s
 42278 11925     cutoff  125        75.00000   76.00000  1.33%   242  436s
 43627 11540   76.00000  105   99   75.00000   76.00000  1.33%   247  449s
 44256 11503   76.00000  116  118   75.00000   76.00000  1.33%   249  461s
 45788 11109   76.00000  112  167   75.00000   76.00000  1.33%   253  473s
 46458 10975   76.00000  114  168   75.00000   76.00000  1.33%   259  485s
 47262 10794   76.00000  135  184   75.00000   76.00000  1.33%   262  498s
 47919 10655     cutoff  144        75.00000   76.00000  1.33%   265  510s
 49094 10327     cutoff  127        75.00000   76.00000  1.33%   268  533s
 49271 10626   76.00000  132  148   75.00000   76.00000  1.33%   270  545s
 50203 10574   76.00000  

 113953  9837     cutoff  152        75.00000   76.00000  1.33%   339 1671s
 114608  9643   76.00000  162  217   75.00000   76.00000  1.33%   340 1692s
 115662  9307     cutoff  144        75.00000   76.00000  1.33%   342 1713s
 116457  9119   76.00000  150  187   75.00000   76.00000  1.33%   343 1739s
 117320  8877     cutoff  143        75.00000   76.00000  1.33%   345 1765s
 118619  8422     cutoff  148        75.00000   76.00000  1.33%   347 1786s
 119051  8347   76.00000  135  194   75.00000   76.00000  1.33%   348 1811s
 120219  7982     cutoff  154        75.00000   76.00000  1.33%   350 1833s
 120987  7780     cutoff  140        75.00000   76.00000  1.33%   351 1857s
 121849  7547   76.00000  148  100   75.00000   76.00000  1.33%   352 1880s
 123231  7097   76.00000  151  192   75.00000   76.00000  1.33%   354 1901s
 123722  6940   76.00000  131  170   75.00000   76.00000  1.33%   355 1924s
 124898  6564   76.00000  139  161   75.00000   76.00000  1.33%   356 1955s
 125611  630

## Entrada de datos: Instancia 3x150x5x1

In [72]:
datafile="3x150x5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [73]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x98306cbe
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3425434 rows and 198057 columns (presolve time = 10s) ...
Presolve removed 3441191 rows and 198081 columns (presolve time = 15s) ...
Presolve removed 3447896 rows and 198081 columns (presolve time = 20s) ...
Presolve removed 3452548 rows and 198081 columns (presolve time =

     0     0   68.37658    0  251   67.00000   68.37658  2.05%     -   64s
     0     0   68.37654    0  244   67.00000   68.37654  2.05%     -   65s
     0     0   68.36561    0  229   67.00000   68.36561  2.04%     -   65s
     0     0   68.36346    0  233   67.00000   68.36346  2.04%     -   65s
     0     0   68.36329    0  253   67.00000   68.36329  2.03%     -   65s
     0     0   68.36303    0  240   67.00000   68.36303  2.03%     -   65s
     0     0   68.36294    0  249   67.00000   68.36294  2.03%     -   65s
     0     0   68.36122    0  234   67.00000   68.36122  2.03%     -   65s
     0     0   68.36076    0  251   67.00000   68.36076  2.03%     -   65s
     0     0   68.36058    0  257   67.00000   68.36058  2.03%     -   65s
     0     0   68.36052    0  247   67.00000   68.36052  2.03%     -   65s
     0     0   68.35932    0  244   67.00000   68.35932  2.03%     -   65s
     0     0   68.35731    0  249   67.00000   68.35731  2.03%     -   65s
     0     0   68.35687  

## Entrada de datos: Instancia 3x150x5x2

In [74]:
datafile="3x150x5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [75]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xa5819e3f
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 5s) ...
Presolve removed 3433924 rows and 198243 columns (presolve time = 10s) ...
Presolve removed 3445252 rows and 198261 columns (presolve time = 15s) ...
Presolve removed 3451320 rows and 198261 columns (presolve time = 20s) ...
Presolve removed 3455421 rows and 198261 columns (presolve time =

## Entrada de datos: Instancia 3x150x5x3

In [76]:
datafile="3x150x5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [77]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x015c30ca
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3432120 rows and 198138 columns (presolve time = 11s) ...
Presolve removed 3445698 rows and 198171 columns (presolve time = 15s) ...
Presolve removed 3452342 rows and 198171 columns (presolve time = 20s) ...
Presolve removed 3457071 rows and 198171 columns (presolve time =

 34413  7251   68.00000   58  206   67.00000   68.00000  1.49%   286  551s
 34500  7265     cutoff   60        67.00000   68.00000  1.49%   286  565s
 34583  7385   68.00000   61  166   67.00000   68.00000  1.49%   286  572s
 35004  7436   68.00000   57  179   67.00000   68.00000  1.49%   287  578s
 35716  7338   68.00000   72  158   67.00000   68.00000  1.49%   288  585s
 36431  7214     cutoff   64        67.00000   68.00000  1.49%   291  593s
 37350  7130     cutoff   63        67.00000   68.00000  1.49%   292  600s
 38104  6880   68.00000   78   38   67.00000   68.00000  1.49%   293  663s
 38110  6884   68.00000  102  221   67.00000   68.00000  1.49%   293  665s
 38114  6887   68.00000   52  186   67.00000   68.00000  1.49%   293  670s
 38119  6890   68.00000   72  155   67.00000   68.00000  1.49%   293  675s
 38123  6893   68.00000   83  281   67.00000   68.00000  1.49%   293  680s
 38136  6903   68.00000   55  104   67.00000   68.00000  1.49%   294  686s
 38142  6907   68.00000  

 117972 17635     cutoff  106        67.00000   68.00000  1.49%   304 1494s
 119082 17677     cutoff  103        67.00000   68.00000  1.49%   305 1504s
 119509 17777     cutoff   94        67.00000   68.00000  1.49%   305 1515s
 120603 17831     cutoff   97        67.00000   68.00000  1.49%   306 1530s
 121202 17901   68.00000   98  133   67.00000   68.00000  1.49%   306 1547s
 121496 17936   68.00000  103  182   67.00000   68.00000  1.49%   306 1557s
 122693 18061   68.00000   92  152   67.00000   68.00000  1.49%   306 1575s
 123505 18180     cutoff   98        67.00000   68.00000  1.49%   307 1585s
 124555 18226   68.00000   99  179   67.00000   68.00000  1.49%   307 1595s
 124913 18345   68.00000   72  113   67.00000   68.00000  1.49%   308 1606s
 126095 18449     cutoff   93        67.00000   68.00000  1.49%   308 1619s
 126726 18639   68.00000  110  171   67.00000   68.00000  1.49%   308 1630s
 127845 18757   68.00000   96  148   67.00000   68.00000  1.49%   309 1640s
 128925 1883

 219762 32092   68.00000   96  120   67.00000   68.00000  1.49%   328 2779s
 221121 32223   68.00000   91  160   67.00000   68.00000  1.49%   328 2786s
 222042 32233   68.00000  120  106   67.00000   68.00000  1.49%   328 2795s
 222162 32286   68.00000  121   86   67.00000   68.00000  1.49%   328 2803s
 222835 32380   68.00000  104  143   67.00000   68.00000  1.49%   328 2810s
 223925 32425     cutoff  103        67.00000   68.00000  1.49%   328 2827s
 224261 32519   68.00000  105  115   67.00000   68.00000  1.49%   328 2835s
 225338 32626   68.00000  114  113   67.00000   68.00000  1.49%   328 2844s
 226457 32760   68.00000  107  123   67.00000   68.00000  1.49%   328 2852s
 227444 32793     cutoff   95        67.00000   68.00000  1.49%   328 2860s
 228431 32895     cutoff   85        67.00000   68.00000  1.49%   328 2868s
 229390 33023   68.00000   97  169   67.00000   68.00000  1.49%   327 2876s
 230414 33120     cutoff  103        67.00000   68.00000  1.49%   327 2884s
 231348 3326

## Entrada de datos: Instancia 3x150x5x4

In [78]:
datafile="3x150x5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [79]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0x9a8b06db
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3436213 rows and 198411 columns (presolve time = 11s) ...
Presolve removed 3449181 rows and 198441 columns (presolve time = 15s) ...
Presolve removed 3456925 rows and 198441 columns (presolve time = 20s) ...
Presolve removed 3462009 rows and 198441 columns (presolve time =

## Entrada de datos: Instancia 3x150x5x5

In [80]:
datafile="3x150x5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [81]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 3
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 3511800 rows, 202500 columns and 22646400 nonzeros
Model fingerprint: 0xdca4e251
Variable types: 0 continuous, 202500 integer (202500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 33975 columns (presolve time = 6s) ...
Presolve removed 3432188 rows and 198126 columns (presolve time = 11s) ...
Presolve removed 3451390 rows and 198156 columns (presolve time = 15s) ...
Presolve removed 3460676 rows and 198156 columns (presolve time = 20s) ...
Presolve removed 3465966 rows and 198156 columns (presolve time =

     0     0   67.25735    0  166   66.00000   67.25735  1.91%     -   53s
     0     0   67.25733    0  157   66.00000   67.25733  1.91%     -   53s
     0     0   67.25592    0  171   66.00000   67.25592  1.90%     -   53s
     0     0   67.25589    0  152   66.00000   67.25589  1.90%     -   53s
     0     0   67.25564    0  150   66.00000   67.25564  1.90%     -   53s
     0     0   67.25553    0  148   66.00000   67.25553  1.90%     -   53s
     0     0   67.25432    0  155   66.00000   67.25432  1.90%     -   53s
     0     0   67.25432    0  112   66.00000   67.25432  1.90%     -   53s
     0     0   67.25432    0  106   66.00000   67.25432  1.90%     -   53s
     0     0   67.25432    0  138   66.00000   67.25432  1.90%     -   54s
     0     0   67.25432    0   34   66.00000   67.25432  1.90%     -   54s
     0     0   67.25432    0   87   66.00000   67.25432  1.90%     -   54s
     0     0   67.25432    0   34   66.00000   67.25432  1.90%     -   54s
     0     0   67.25432  